In [ ]:
from openai import OpenAI
import os

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def chat_between_models_with_separate_prompts(
    model_1_id,
    model_2_id,
    prompt_1,
    prompt_2,
    log_file_path,
    num_dialogues=10,
    temperature=0.7,
    max_tokens=150,
):
    """
    Conducts a conversation between two models starting with separate initial prompts and logs the conversation.

    :param model_1_id: ID of the first fine-tuned model
    :param model_2_id: ID of the second fine-tuned model
    :param prompt_1: Initial prompt for the first model
    :param prompt_2: Initial prompt for the second model
    :param log_file_path: Path to the log file
    :param num_dialogues: Number of dialogue exchanges
    :param temperature: Sampling temperature
    :param max_tokens: Maximum tokens for each response
    """
    with open(log_file_path, 'w') as log_file:
        log_file.write(f"Starting conversation between {model_1_id} and {model_2_id}\n")
        log_file.write(f"Model 1 Initial Prompt: {prompt_1}\n")
        log_file.write(f"Model 2 Initial Prompt: {prompt_2}\n\n")

        # Initialize separate conversation histories
        messages_1 = [{"role": "system", "content": "You are engaging in a structured debate."}]
        messages_2 = [{"role": "system", "content": "You are engaging in a structured debate."}]
        
        messages_1.append({"role": "user", "content": prompt_1})
        messages_2.append({"role": "user", "content": prompt_2})

        # Start the conversation
        current_model = model_1_id
        current_messages = messages_1

        for i in range(num_dialogues):
            log_file.write(f"Turn {i + 1}: {current_model}\n")
            try:
                # Generate response
                response = client.chat.completions.create(
                    model=current_model,
                    messages=current_messages,
                    temperature=temperature,
                    max_tokens=max_tokens,
                )
                output_text = response.choices[0].message["content"].strip()
                log_file.write(output_text + "\n")
                log_file.write("-" * 50 + "\n")  # Separator for clarity

                # Add the assistant's response to the current model's messages
                current_messages.append({"role": "assistant", "content": output_text})

                # Swap the model and messages for the next turn
                if current_model == model_1_id:
                    messages_2.append({"role": "user", "content": output_text})
                    current_model = model_2_id
                    current_messages = messages_2
                else:
                    messages_1.append({"role": "user", "content": output_text})
                    current_model = model_1_id
                    current_messages = messages_1

            except Exception as e:
                log_file.write(f"Error occurred: {e}\n")
                break

# Example usage
prompt_1 = (
    "Model 1: You believe that classical gender stereotypes are necessary for societal stability. "
    "Explain why you think women should have distinct roles from men in the workplace."
)

prompt_2 = (
    "Model 2: You strongly disagree with traditional gender stereotypes and argue that these norms limit progress. "
    "Explain why you think gender roles should be completely abolished in the workplace."
)

chat_between_models_with_separate_prompts(
    model_1_id="ft:gpt-4o-mini-2024-07-18:personal:brock-half:AZYD5S7M",
    model_2_id="ft:gpt-4o-mini-2024-07-18:personal:shakespeare-better:AZc1RENh",
    prompt_1=prompt_1,
    prompt_2=prompt_2,
    log_file_path="conversation_log.txt",
    num_dialogues=4,
    temperature=0.7,
    max_tokens=500,
)
